# Model Signature

## Column Based Signature

In [ ]:
import mlflow
from mlflow_for_ml_dev.src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(uri=(get_project_root() / 'mlruns').as_uri())

## Defining a signature

In [2]:
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema 
from mlflow.types.schema import ColSpec

from sklearn.datasets import load_iris

In [3]:
# loading the iris dataset
iris = load_iris(as_frame=True)
iris.data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [4]:
# target
iris.target

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

In [5]:
# schema for the feature columns
col = ColSpec(type="double", name="sepal length (cm)", required=True)
col.to_dict()

{'type': 'double', 'name': 'sepal length (cm)', 'required': True}

In [6]:
col_specifications = [
    ColSpec(type="double", name=feature_name, required=True) for feature_name in iris.feature_names
    ]
model_input = Schema(inputs = col_specifications)

In [7]:
model_output = Schema(inputs= [ColSpec(type="integer", name="species", required=True)])

In [8]:
model_output.to_dict()
model_input.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [9]:
# creating the model signature
model_signature = ModelSignature(
    inputs = model_input,
    outputs= model_output
)

In [10]:
from pprint import pprint

pprint(model_signature.to_dict(), indent=2)

{ 'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
            'true}, {"type": "double", "name": "sepal width (cm)", "required": '
            'true}, {"type": "double", "name": "petal length (cm)", '
            '"required": true}, {"type": "double", "name": "petal width (cm)", '
            '"required": true}]',
  'outputs': '[{"type": "integer", "name": "species", "required": true}]',
  'params': None}


In [11]:
model_signature.inputs.to_dict()

[{'type': 'double', 'name': 'sepal length (cm)', 'required': True},
 {'type': 'double', 'name': 'sepal width (cm)', 'required': True},
 {'type': 'double', 'name': 'petal length (cm)', 'required': True},
 {'type': 'double', 'name': 'petal width (cm)', 'required': True}]

In [12]:
model_signature.outputs.to_dict()

[{'type': 'integer', 'name': 'species', 'required': True}]

In [13]:
model_signature.params == None

True

### Example: Login model using schema

In [14]:
from sklearn.ensemble import RandomForestClassifier 

rfc = RandomForestClassifier()
rfc.fit(iris.data, iris.target)

with mlflow.start_run(run_name="model_signature") as run:
    mlflow.sklearn.log_model(rfc, "rfc", signature=model_signature)

In [15]:
my_model = mlflow.sklearn.load_model("runs:/"+run.info.run_id+"/rfc")
my_model.predict(iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])